## Import Packages

In [11]:
import sqlite3
from sqlite3 import Error
import pandas as pd
database_file_location = "./data/suicide.db"

## Collect data into dataframe of format:
* columns = ["state", "s2019", "s2020"]
* index = "state"

In [12]:
df2019_suicides = pd.read_json('./data/properties_2019.json')
df2020_suicides = pd.read_json('./data/properties_2020.json')

In [13]:
df2020_suicides.head()

,ADMIN,ISO_A3,ISO_A2,year,s2019,s2020
0,Aruba,ABW,AW,2020,56,89
1,Afghanistan,AFG,AF,2020,52,61
2,Angola,AGO,AO,2020,71,53
3,Anguilla,AIA,AI,2020,48,32
4,Albania,ALB,AL,2020,66,85


In [14]:
df2019_suicides.head()

,ADMIN,ISO_A3,ISO_A2,year,s2019
0,Aruba,ABW,AW,2019,56
1,Afghanistan,AFG,AF,2019,52
2,Angola,AGO,AO,2019,71
3,Anguilla,AIA,AI,2019,48
4,Albania,ALB,AL,2019,66


In [15]:
# df2020_suicides.rename(columns={'value': "s2020"}, inplace=True)
# df2019_suicides.rename(columns={'value': "s2019"}, inplace=True)

In [16]:
df2020 = df2020_suicides[['ADMIN', 's2020']]
df2019 = df2019_suicides[['ADMIN', 's2019']]

In [17]:
suicides_by_year_df = df2020.merge(df2019)
suicides_by_year_df.set_index("ADMIN", inplace=True)
suicides_by_year_df.head()

,s2020,s2019
ADMIN,,
Aruba,89,56
Afghanistan,61,52
Angola,53,71
Anguilla,32,48
Albania,85,66


In [18]:
# more fake data
suicides_by_year_df['s2018'] = (suicides_by_year_df['s2019'] + suicides_by_year_df['s2020'])/2
suicides_by_year_df['s2017'] = abs(suicides_by_year_df['s2019'] - suicides_by_year_df['s2020'])*4
suicides_by_year_df.head()

,s2020,s2019,s2018,s2017
ADMIN,,,,
Aruba,89,56,72.5,132
Afghanistan,61,52,56.5,36
Angola,53,71,62.0,72
Anguilla,32,48,40.0,64
Albania,85,66,75.5,76


In [27]:
suicides_T = suicides_by_year_df.T

In [28]:
suicides_T.to_json('./data/properties4.json')

## Add records to database

In [19]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

In [23]:
def create_record(conn, project):
    """
    Create a new project into the projects table
    :param conn:
    :param project:
    :return: project id
    """
    sql = ''' INSERT INTO suicides(country,s2017,s2018,s2019,s2020)
              VALUES(?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, project)
    conn.commit()
    return cur.lastrowid

In [24]:
def main():
    database = database_file_location
    
    # create a database connection
    conn = create_connection(database)
    
    for row in suicides_by_year_df.itertuples():
        with conn:
            # create a new record
            record = (row[0], row[4], row[3], row[2], row[1]);
            record_id = create_record(conn, record)


In [25]:
if __name__ == '__main__':
    main()